In [1]:
import os
import sys

if os.path.basename(os.getcwd()) == "notebooks":
    os.chdir("..")

sys.path.append("source")

In [2]:
import numpy as np
import torch
from aim import Repo
from model import VAE
from sklearn.metrics import (
    calinski_harabasz_score,
    davies_bouldin_score,
    silhouette_score,
)
from torch.utils.data import ConcatDataset, DataLoader
from torchvision.datasets import MNIST, FashionMNIST
from torchvision.transforms import v2
from tqdm import tqdm

In [3]:
repo = Repo(".")

torch.manual_seed(123)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [4]:
def get_dataloader(dataset, batch_size, num_workers, device):
    match dataset:
        case "mnist":
            transform = v2.Compose(
                [
                    v2.ToImage(),
                    v2.ToDtype(torch.float32, scale=True),
                    v2.Lambda(lambda x: torch.flatten(x)),
                ]
            )
            train_dataset = MNIST(
                root=".data", train=True, download=True, transform=transform
            )
            valid_dataset = MNIST(
                root=".data", train=False, download=True, transform=transform
            )
            dataset = ConcatDataset([train_dataset, valid_dataset])

        case "fashion-mnist":
            transform = v2.Compose(
                [
                    v2.ToImage(),
                    v2.ToDtype(torch.float32, scale=True),
                    v2.Lambda(lambda x: torch.flatten(x)),
                ]
            )
            train_dataset = FashionMNIST(
                root=".data", train=True, download=True, transform=transform
            )
            valid_dataset = FashionMNIST(
                root=".data", train=False, download=True, transform=transform
            )
            dataset = ConcatDataset([train_dataset, valid_dataset])

    dataloader = DataLoader(
        dataset,
        shuffle=True,
        batch_size=batch_size,
        num_workers=num_workers,
        pin_memory=device.type == "cuda",
    )

    return dataloader


def get_runs(repo, dataset):
    return list(
        filter(
            lambda run: run["hparams"]["dataset"] == dataset,
            repo.iter_runs(),
        )
    )


def get_model(run, device):
    model = VAE(
        run["hparams"]["input_dim"],
        run["hparams"]["latent_dim"],
        run["hparams"]["kl_lambda"],
    ).to(device)
    model.load_state_dict(torch.load(f"weights/{run.hash}_200.pth", device))
    return model


def calc_outputs(model, dataloader, device):
    model.eval()
    outputs = []
    with torch.no_grad():
        for x, y in dataloader:
            x = x.to(device)
            mu_z, log_sigma2_z = model.encoder(x)
            z = model.sample_z(mu_z, log_sigma2_z)
            z = z.numpy(force=True)
            y = y.numpy(force=True)
            outputs.append((z, y))
    z = np.concatenate([output[0] for output in outputs])
    y = np.concatenate([output[1] for output in outputs])
    return z, y


def print_results_1(results):
    for key, value in results.items():
        value = list(value.values())
        print(
            f"{key}:\n\t{np.mean(value):.2f}±{np.std(value):.2f} ({np.min(value):.2f} - {np.max(value):.2f})"
        )


def print_results_2(results):
    for key, value in results.items():
        print(f"{key}:\n\t{max(value, key=value.get)}↑\n\t{min(value, key=value.get)}↓")

## 2. Unsupervised metrics

### MNIST

In [5]:
dataloader = get_dataloader("mnist", 1_000, 4, device)

In [6]:
results = {}
for run in tqdm(get_runs(repo, "mnist")):
    model = get_model(run, device)
    z, y = calc_outputs(model, dataloader, device)

    results.setdefault("Silhouette coefficient", {})[run.hash] = silhouette_score(
        z, y, sample_size=7_000, random_state=123
    )
    results.setdefault("Calinski-Harabasz index", {})[run.hash] = (
        calinski_harabasz_score(z, y)
    )
    results.setdefault("Davies-Bouldin index", {})[run.hash] = davies_bouldin_score(
        z, y
    )

  0%|          | 0/10 [00:00<?, ?it/s]

100%|██████████| 10/10 [01:07<00:00,  6.73s/it]


In [7]:
results

{'Silhouette coefficient': {'20b55385c51f4c54b551073d': 0.03959396,
  '4391c7c276864281a53de508': 0.042731095,
  '5c07daaa8c5e4138843637e8': 0.04062405,
  '7d446d1067304739b6308f5f': 0.02742201,
  'a9b5c6765750470dae0bdc34': 0.037806276,
  'ab232011b7ab41c0bb407ed5': 0.045708757,
  'd1fbb775d5164b4693e612cd': 0.03202333,
  'd3c6ed502a3241cf964aa494': 0.036055494,
  'd7044fcfb439459c86296e5a': 0.03362379,
  'e42e6553aa35426d838bb824': 0.032474115},
 'Calinski-Harabasz index': {'20b55385c51f4c54b551073d': 2663.645180403851,
  '4391c7c276864281a53de508': 2637.4471739339983,
  '5c07daaa8c5e4138843637e8': 2756.2493565280374,
  '7d446d1067304739b6308f5f': 2388.506444159518,
  'a9b5c6765750470dae0bdc34': 2587.488652935579,
  'ab232011b7ab41c0bb407ed5': 2714.7262450906796,
  'd1fbb775d5164b4693e612cd': 2571.2503015402503,
  'd3c6ed502a3241cf964aa494': 2653.827761463934,
  'd7044fcfb439459c86296e5a': 2674.0683630673993,
  'e42e6553aa35426d838bb824': 2501.5036382490584},
 'Davies-Bouldin index':

In [8]:
print_results_1(results)

Silhouette coefficient:
	0.04±0.01 (0.03 - 0.05)
Calinski-Harabasz index:
	2614.87±102.28 (2388.51 - 2756.25)
Davies-Bouldin index:
	3.33±0.27 (3.10 - 4.10)


In [9]:
print_results_2(results)

Silhouette coefficient:
	ab232011b7ab41c0bb407ed5↑
	7d446d1067304739b6308f5f↓
Calinski-Harabasz index:
	5c07daaa8c5e4138843637e8↑
	7d446d1067304739b6308f5f↓
Davies-Bouldin index:
	7d446d1067304739b6308f5f↑
	20b55385c51f4c54b551073d↓


In [10]:
list(results["Silhouette coefficient"].keys())[
    np.argmax(
        (
            np.argsort(
                np.argsort(np.array(list(results["Silhouette coefficient"].values())))
            )
            + np.argsort(
                np.argsort(np.array(list(results["Calinski-Harabasz index"].values())))
            )
            + np.argsort(
                np.argsort(-np.array(list(results["Davies-Bouldin index"].values())))
            )
        )
    )
]

'5c07daaa8c5e4138843637e8'

### Fashion MNIST

In [11]:
dataloader = get_dataloader("fashion-mnist", 1_000, 4, device)

In [12]:
results = {}
for run in tqdm(get_runs(repo, "fashion-mnist")):
    model = get_model(run, device)
    z, y = calc_outputs(model, dataloader, device)

    results.setdefault("Silhouette coefficient", {})[run.hash] = silhouette_score(
        z, y, sample_size=7_000, random_state=123
    )
    results.setdefault("Calinski-Harabasz index", {})[run.hash] = (
        calinski_harabasz_score(z, y)
    )
    results.setdefault("Davies-Bouldin index", {})[run.hash] = davies_bouldin_score(
        z, y
    )

100%|██████████| 10/10 [01:12<00:00,  7.22s/it]


In [13]:
results

{'Silhouette coefficient': {'26e2e3ad8f1b41889b29a8df': 0.016106013,
  '28675bfcab8c435cbf5c9e8d': 0.017914977,
  '4cb62b83ce6247fa81b8e956': 0.019700032,
  '7fcd1ff5b88344d59fe86bd2': 0.022270354,
  '8e16e0b3b08646d49107603c': 0.025249556,
  'a409a4518e584f7b9db15168': 0.019773168,
  'aa6a2c3896ae4244af0ab960': 0.021436794,
  'c5a1111d61554d83b153dbee': 0.021726428,
  'd71c33363c314802bf8df237': 0.016524514,
  'f81570fee9104f3c984bbf5f': 0.016168583},
 'Calinski-Harabasz index': {'26e2e3ad8f1b41889b29a8df': 1854.1736792193512,
  '28675bfcab8c435cbf5c9e8d': 1846.0673735678122,
  '4cb62b83ce6247fa81b8e956': 1963.7472823807864,
  '7fcd1ff5b88344d59fe86bd2': 2012.6794636153638,
  '8e16e0b3b08646d49107603c': 2002.3058637578365,
  'a409a4518e584f7b9db15168': 1929.5159828373573,
  'aa6a2c3896ae4244af0ab960': 1935.5990041079428,
  'c5a1111d61554d83b153dbee': 1939.3939280716504,
  'd71c33363c314802bf8df237': 1850.5971273149898,
  'f81570fee9104f3c984bbf5f': 1891.7006336480297},
 'Davies-Bouldi

In [14]:
print_results_1(results)

Silhouette coefficient:
	0.02±0.00 (0.02 - 0.03)
Calinski-Harabasz index:
	1922.58±57.74 (1846.07 - 2012.68)
Davies-Bouldin index:
	4.32±0.11 (4.09 - 4.48)


In [15]:
print_results_2(results)

Silhouette coefficient:
	8e16e0b3b08646d49107603c↑
	26e2e3ad8f1b41889b29a8df↓
Calinski-Harabasz index:
	7fcd1ff5b88344d59fe86bd2↑
	28675bfcab8c435cbf5c9e8d↓
Davies-Bouldin index:
	d71c33363c314802bf8df237↑
	8e16e0b3b08646d49107603c↓


In [16]:
list(results["Silhouette coefficient"].keys())[
    np.argmax(
        (
            np.argsort(
                np.argsort(np.array(list(results["Silhouette coefficient"].values())))
            )
            + np.argsort(
                np.argsort(np.array(list(results["Calinski-Harabasz index"].values())))
            )
            + np.argsort(
                np.argsort(-np.array(list(results["Davies-Bouldin index"].values())))
            )
        )
    )
]

'8e16e0b3b08646d49107603c'